## Practice running the Permafrost Discovery Gateway visualization workflow

- This notebook is designed to be run on the NCEAS Datateam server, because that is where the ice-wedge polygon data is archived.
- Create a fresh Python environment and install the requirements with `pip install -r requirements.txt`
- Execute staging, rasterization, and web-tiling steps.
- Process 3 adjacent ice-wedge polygon files from [Wrangle Island](https://www.google.com/maps/place/Wrangel+Island/@71.3497058,179.8238705,9z/data=!4m6!3m5!1s0x50a70636a5f5033f:0xe1dca925085b4bc3!8m2!3d71.2488724!4d-179.9789208!16zL20vMDMyZnRq), off the coast of Russia
- After running through these steps in chunks in this notebook, it's a great idea to transfer the code to a script and run as a `tmux` session. See `workflow.py` for an example.

In [ ]:
# filepaths
from pathlib import Path
import os

# visual checks & vector data wrangling
import geopandas as gpd

# staging
import pdgstaging
from pdgstaging import TileStager

# rasterization & web-tiling
import pdgraster
from pdgraster import RasterTiler

# logging
from datetime import datetime
import logging
import logging.handlers
from pdgstaging import logging_config

### Import 3 adjacent data files with ice-wedge polygon detections

In [ ]:
data_dir = '/home/jcohen/iwp_russia_subset_clipToFP_PR/'
base_dir = Path(data_dir + 'iwp')
filename = '*.shp'
# To define each .shp file within each subdir as a string representation with forward slashes, use as_posix()
# The ** represents that any subdir string can be present between the base_dir and the filename
input = [p.as_posix() for p in base_dir.glob('**/' + filename)]
input

### Import the footprint files for the ice-wedge polygon detections

These files each represent the bounding box of the respective ice-wedge polygon detections file. This is used for deduplication where the scenes overlap.

In [ ]:
# pull filepaths for footprints in the same way we pulled IWP shp file paths
base_dir_fp = Path(data_dir + 'footprints')
# To define each .shp file within each subdir as a string representation with forward slashes, use as_posix()
# The ** represents that any subdir string can be present between the base_dir and the filename
fps = [p.as_posix() for p in base_dir_fp.glob('**/' + filename)]
fps

### Notes on logging

- Logging for the workflow is configured by the file `logging_config.py` within the `pdgstaging` package. We imported this configuration at the start of this notebook.
- The written logging file `log.log` is written to the `/tmp` directory on Datateam. It's good practice to `mv` this file from `/tmp` to your working directory after each workflow run. In the script that runs this workflow, `workflow.py`, this command is added to the end so we don't forget. If the file is not moved, logging statements from the next run are simply appended.
- When troubleshooting, it's helpful to ctrl + f for certain logged statements when troubleshooting. For example, if fewer files were staged than expected, you can search for "error" or "failed". If you are debugging a silent error and suspect that the issue has something to do with the order in which input files are processed, you can search for the input filenames to determine which was staged first.

#### Plot the files with their footprints

Each of the following plots takes ~2-5 minutes to generate on Datateam.

In [ ]:
# first pair: one file of IWP detections and the associated footprint file 
shp = gpd.read_file(input[0])
footprint = gpd.read_file(fps[0])
ax = shp.plot(color='none', edgecolor='green', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# second pair
shp = gpd.read_file(input[1])
footprint = gpd.read_file(fps[1])
ax = shp.plot(color='none', edgecolor='blue', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# third pair
shp = gpd.read_file(input[2])
footprint = gpd.read_file(fps[2])
ax = shp.plot(color='none', edgecolor='green', linewidths=1.5, figsize=(14,14))
footprint.plot(ax=ax, color='none', edgecolor='black', linewidths=0.5, figsize=(14,14))

In [ ]:
# plot 2 non-coastal adjacent iwp files together, no footprints
shp1 = gpd.read_file(input[1])
shp2 = gpd.read_file(input[2])
ax = shp1.plot(color='none', edgecolor='blue', linewidths=1.5, figsize=(14,14), alpha=0.3)
shp2.plot(ax=ax, color='none', edgecolor='green', linewidths=0.5, figsize=(14,14), alpha=0.3)

### Define the configuration and stager

The configuration defines how the workflow is executed, such as if we deduplicate polygons, _how_ we deduplicate, the color palette, statistics that become bands in the output geoTIFFFs, etc. You can define the config in a number of ways:
1. input the config directly to `TileStager()`
2. _or_ define the config as an object in the script, and feed it into `TileStager` with `pdgstaging.TileStager(config)`, as we do in the next chunk. This is preferred over option 1 because we feed the same config into the `RasterTiler` later, and option 2 allows for the config to only be defined once.
3. _or_ define the config within a separate `.py` or `.json` script and source it in with the following:

    ```python
    # config.py must be a script in same level of folder hierachy as this notebook 
    # that contains config defined as an object like `config = {...}` 
    import config 
    config = config.config
    pdgstaging.TileStager(config)
    ```

#### Notes on configuration: 
- Many of the settings in the config specified below are defaults. You can find default values for the config [here](https://github.com/PermafrostDiscoveryGateway/viz-staging/blob/4f31e951600d54c128f76b48a47ec390261fb548/pdgstaging/ConfigManager.py#L351-L422).
- The statistic(s) specified are calculated during rasterization, and each stat each becomes a band. You can add a separate stat, remove a stat, or change the details of them as desired. The "name" of each statistic can be anything, but the "property" needs to be either found within the vector data itself or be one of the custom stats defined in `pdgstaging`. Here, we use `area_per_pixel_area`, which is a custom one.
- Color palette can be changed, but must be at least 2 values.
- Input data types for the staging step should be vectors, like `.shp` or `.gpkg`. We specify `.shp` below because the IWP input data are shapefiles. 

For more information about the configuration, there is extensive documentation in [PermafrostDiscoveryGateway/viz-staging/pdgstaging/ConfigManager.py](https://github.com/PermafrostDiscoveryGateway/viz-staging/blob/main/pdgstaging/ConfigManager.py)


In [ ]:
config = {
  "deduplicate_clip_to_footprint": True, 
  "dir_input": data_dir + "iwp/", 
  "ext_input": ".shp",
  "ext_footprints": ".shp",
  "dir_footprints": data_dir + "footprints/", 
  "dir_staged": "staged/",
  "dir_geotiff": "geotiff/", 
  "dir_web_tiles": "web_tiles/", 
  "filename_staging_summary": "staging_summary.csv",
  "filename_rasterization_events": "raster_events.csv",
  "filename_rasters_summary": "raster_summary.csv",
  "filename_config": "config",
  "simplify_tolerance": 0.1,
  "tms_id": "WGS1984Quad",
  "z_range": [
    0,
    15
  ],
  "geometricError": 57,
  "z_coord": 0,
  "statistics": [
    {
      "name": "iwp_coverage",
      "weight_by": "area",
      "property": "area_per_pixel_area",
      "aggregation_method": "sum",
      "resampling_method": "average",
      "val_range": [
        0,
        1
      ],
      "palette": [
        "#66339952",
        "#ffcc00"
      ],
      "nodata_val": 0,
      "nodata_color": "#ffffff00"
    },
  ],
  "deduplicate_at": [
    "raster"
  ],
  "deduplicate_keep_rules": [
    [
      "Date",
      "larger"
    ]
  ],
  "deduplicate_method": "footprints"
}

In [ ]:
stager = TileStager(config)

## Staging

The following chunk will create a new directory called `staged` and populate it with subdirectories and geopackage files. Each file is a tile in the Tile Matrix Set `WGS1984Quad`.

In [ ]:
stager.stage_all()

Number of `staged` files produced: **3214**

## Rasterization & web-tiling

- Use the `pdgraster.RasterTiler()` function to create the `rasterizer`, then use it to execute the rasterize function: `rasterizer.rasterize_vectors()`.
- Alternatively, can run these two steps as one with just `pdgraster.RasterTiler().rasterize_all()`. Just as in staging, you can input the config directly to `RasterTiler()`, or save the config as an object earlier in the script, or save the config as a separate script and source it in. 
- `rasterize_all()` is a wrapper for `rasterize_vectors()`. It pulls all staged filepaths from the staged dir and rasterizes all z-levels . We do _not_ use `rasterize_all()` when rasterizing in parallel with `parsl` or `ray`. It is exclusively used for small datasets that are not run in parallel. We have to create specific `@parsl` or `@ray.remote` wrapper functions around `stage()` and `rasterize_vectors()` if using those packages for staging, rasterizaiton, etc. For an example, see [here](https://github.com/PermafrostDiscoveryGateway/viz-workflow/blob/8c1997a9d2456bcb79ba1b3ab0f82b3b2b30b141/IN_PROGRESS_VIZ_WORKFLOW.py#L668-L693) for how we rasterize in the ray workflow.
- Rasterization when executed with `rasterize_all()` creates `.tif` files in the output `geotiff` dir, _and_ creates the same number of `.png` files in the output `web_tiles` dir.
    - When we use `rasterize_vectors()` instead, we _only create the `.tif` files and not the `.png` files_. So that needs to be executed as a separate step with `rasterizer.webtiles_from_geotiffs()` after the `rasterize_vectors() step`, and _in between those steps_ we need to manually "update the ranges" in the rasterizer to ensure that the colors within one z-level of `.png` files looks appropriate when visualized in the portal. We will cross that bridge as necessay, just as we do [here](https://github.com/PermafrostDiscoveryGateway/viz-workflow/blob/8c1997a9d2456bcb79ba1b3ab0f82b3b2b30b141/IN_PROGRESS_VIZ_WORKFLOW.py#L592) in the `ray` workflow on Delta.
- The number of z-level 15 tiles in the `staged` dir should match the number of z-level 15 tiles in the `geotiff` and `web_tiles` dirs. The total number of files in both the `geotiff` and `web_tiles` dirs is a _lot_ more than the number of files in `staged` because `staged` _only contains the highest zoom level_ with no parent z-levels.
- The web tiles are what we actually visualize on the PDG portal and local cesium. We create the rasters for summary stats (the data behind the web tiles, stored in bands of the `.tif` files)




In [ ]:
RasterTiler(config).rasterize_all()

Number of `geotiff` files produced (for all z-levels): **4376**

Number of `web_tiles` produced (for all z-levels): **4376**
  - The number is the same because we only calculated 1 statistic, so one band in each geoTIFF. Each raster band becomes one web tile, so if there are two statistics calculated, there will be twice as many web tiles as there are geoTIFFs.

-----------

You can use `rasterio` to plot the rasters if you want to get an idea what the statistics look like. Change the number to choose which band (statistic) to visualize. Example:

```python
import matplotlib.pyplot as plt
import rasterio

file1 = rasterio.open('/path/to/raster.tif')

plt.imshow(file1.read(1), cma = 'pink')
```

## Visualizing the web tiles

Ypu can simply open the `.png` files to view them one at a time, but it's much better to view them with Cesium! For steps for how to visualize the web tiles with local Cesium, see [documentation here in pdg-info](https://github.com/robyngit/pdg-info/blob/main/05_displaying-the-tiles.md#option-1-run-cesium-locally).